# Model New Data

In [1]:
import numpy as np
import pandas as pd
import acquire
import prepare
import model
from env import api_key

# Using Data At 10 Minute Timeframe

In [2]:
#Load final_10 csv file
data = pd.read_csv('../master10_csv/final_10.csv', index_col = [0])
data

,airdragon_team100,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,...,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,chemtechdragon_team200,riftherald_team200,airdragon_team200,waterdragon_team200
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3033.0,2297.0,3978.0,3193.0,4131.0,2971.0,NaN,NaN,NaN,NaN
1,0.0,1.0,1.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,...,2907.0,2719.0,2691.0,3758.0,4331.0,3662.0,NaN,NaN,NaN,NaN
2,0.0,1.0,2.0,1.0,4.0,5.0,7.0,1.0,3.0,2.0,...,3192.0,3234.0,4260.0,3189.0,4858.0,2538.0,NaN,NaN,NaN,NaN
3,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,2921.0,2417.0,4801.0,4457.0,3837.0,2916.0,1.0,NaN,NaN,NaN
4,1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,...,3005.0,2671.0,4865.0,3579.0,4496.0,2644.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4858,0.0,1.0,1.0,2.0,2.0,1.0,4.0,0.0,1.0,2.0,...,3365.0,2670.0,4031.0,3851.0,2589.0,3621.0,NaN,NaN,NaN,NaN
4859,0.0,3.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,1.0,...,3119.0,4443.0,3667.0,4158.0,2940.0,3527.0,NaN,NaN,NaN,NaN
4860,0.0,2.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0,...,4471.0,2958.0,4988.0,4474.0,2615.0,4185.0,NaN,NaN,NaN,NaN
4861,0.0,0.0,4.0,2.0,1.0,0.0,2.0,0.0,5.0,1.0,...,3347.0,1972.0,5326.0,2685.0,3750.0,3402.0,NaN,NaN,NaN,NaN


In [3]:
data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4863 entries, 0 to 4862
Data columns (total 196 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    airdragon_team100                 float64
 1    assistsplayer_1                   float64
 2    assistsplayer_10                  float64
 3    assistsplayer_2                   float64
 4    assistsplayer_3                   float64
 5    assistsplayer_4                   float64
 6    assistsplayer_5                   float64
 7    assistsplayer_6                   float64
 8    assistsplayer_7                   float64
 9    assistsplayer_8                   float64
 10   assistsplayer_9                   float64
 11   baron_team100                     float64
 12   chemtechdragon_team100            float64
 13   currentGold_1                     float64
 14   currentGold_10                    float64
 15   currentGold_2                     float64
 16   currentGold_3         

In [4]:
#Now prepare it
train, test = prepare.prepare(data)
train.shape, test.shape

((3890, 265), (973, 265))

In [5]:
#Drop all columns that are categorical. They are not useful
cols_to_drop = list(train.select_dtypes('object').columns)

In [6]:
cols_to_drop

['gameMode', 'gameName', 'gameType', 'gameVersion', 'matchId']

In [7]:
train = train.drop(columns = cols_to_drop)
test = test.drop(columns = cols_to_drop)

In [8]:
train.shape, test.shape

((3890, 260), (973, 260))

In [46]:
#Check out the prepared column names
train.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3890 entries, 3513 to 3582
Data columns (total 260 columns):
 #    Column                                    Dtype  
---   ------                                    -----  
 0    BlueTeamAirDragon                         float64
 1    BlueTopAssists                            float64
 2    RedSupportAssists                         float64
 3    BlueJungleAssists                         float64
 4    BlueMidAssists                            float64
 5    BlueADCAssists                            float64
 6    BlueSupportAssists                        float64
 7    RedTopAssists                             float64
 8    RedJungleAssists                          float64
 9    RedMidAssists                             float64
 10   RedADCAssists                             float64
 11   BlueTeamBarons                            float64
 12   BlueTeamChemtechDragon                    float64
 13   BlueTopCurrentGold                        f

In [9]:
#Now separate it into X and y groups
X_train, y_train = train.drop(columns = ['winningTeam']), train.winningTeam
X_test, y_test = test.drop(columns = ['winningTeam']), train.winningTeam

In [51]:
#Create the dict of hyperparameters to optimize across
param_dict = {
    'max_depth': range(1, 16),
    'min_samples_leaf': range(1, 16)
}

In [11]:
#Now build the models
best_model = model.get_random_forest_models(X_train, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.6362
Max Depth:  14
Min Samples Per Leaf:  12


In [12]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

,0
BlueTeamTotalGoldDifference,0.033197
BlueTeamXpDifference,0.031976
RedTeamTotalGoldDifference,0.026750
BlueTeamTotalDmgDifference,0.024324
BlueTeamTotalMinionsMonstersDifference,0.019592
gameDuration,0.018485
RedTeamXp,0.015649
RedTeamTotalGold,0.013054
BlueTeamDeathsDifference,0.011526
BlueTeamXp,0.010815


# Can We Remove Individual Stats and Maintain or Improve Model Accuracy?

In [13]:
#Only keep columns that have the word 'team' in their name
cols_to_keep = X_train.columns[X_train.columns.str.contains('Team')]
cols_to_keep

Index(['BlueTeamAirDragon', 'BlueTeamBarons', 'BlueTeamChemtechDragon',
       'BlueTeamDragons', 'RedTeamDragons', 'BlueTeamEarthDragon',
       'RedTeamEarthDragon', 'BlueTeamFireDragon', 'RedTeamFireDragon',
       'BlueTeamHextechDragon', 'RedTeamHextechDragon', 'RedTeamInhibsLost',
       'BlueTeamRiftHeralds', 'BlueTeamTowersLost', 'RedTeamTowersLost',
       'BlueTeamWaterDragon', 'RedTeamChemtechDragon', 'RedTeamRiftHeralds',
       'RedTeamAirDragon', 'RedTeamWaterDragon', 'BlueTeamDeaths',
       'RedTeamDeaths', 'BlueTeamGoldPerSec', 'RedTeamGoldPerSec',
       'BlueTeamJungleMinionsKilled', 'RedTeamJungleMinionsKilled',
       'BlueTeamKills', 'RedTeamKills', 'BlueTeamLevel', 'RedTeamLevel',
       'BlueTeamMagicDamageDoneToChampions',
       'RedTeamMagicDamageDoneToChampions', 'BlueTeamMinionsKilled',
       'RedTeamMinionsKilled', 'BlueTeamPhysicalDamageDoneToChampions',
       'RedTeamPhysicalDamageDoneToChampions',
       'BlueTeamTimeEnemySpentControlled', 'RedTeamTim

In [14]:
X_train_team_data = X_train.loc[:, cols_to_keep]
X_train_team_data

,BlueTeamAirDragon,BlueTeamBarons,BlueTeamChemtechDragon,BlueTeamDragons,RedTeamDragons,BlueTeamEarthDragon,RedTeamEarthDragon,BlueTeamFireDragon,RedTeamFireDragon,BlueTeamHextechDragon,...,BlueTeamTotalMinionsMonstersDifference,BlueTeamTimeCCingDifference,BlueTeamWaterDragonDifference,BlueTeamAirDragonDifference,BlueTeamChemtechDragonDifference,BlueTeamFireDragonDifference,BlueTeamHextechDragonDifference,BlueTeamEarthDragonDifference,BlueTeamKdaDifference,BlueTeamJungleDiffy
3513,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,12,110674,0.0,0.0,0.0,1.0,0.0,0.0,14.25,100.0
1250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41,76997,0.0,0.0,0.0,0.0,0.0,0.0,9.75,200.0
3532,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,35,-65221,0.0,0.0,0.0,0.0,1.0,0.0,17.25,-12.0
3858,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,34,-24134,0.0,0.0,0.0,-1.0,0.0,0.0,5.25,208.0
528,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-24,-28320,0.0,0.0,0.0,-1.0,0.0,0.0,6.75,111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,64,-31773,0.0,0.0,0.0,0.0,0.0,0.0,20.25,190.0
4060,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,-44,28761,0.0,0.0,0.0,0.0,0.0,-1.0,-21.75,200.0
1346,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,-17,-5678,0.0,0.0,0.0,0.0,0.0,-1.0,-0.75,-205.0
3454,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14,93964,1.0,0.0,0.0,0.0,0.0,0.0,2.25,-204.0


In [15]:
#Build the models
best_model = model.get_random_forest_models(X_train_team_data, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.638
Max Depth:  12
Min Samples Per Leaf:  7


In [16]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train_team_data.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

,0
BlueTeamTotalGoldDifference,0.060294
RedTeamTotalGoldDifference,0.059468
BlueTeamXpDifference,0.049065
BlueTeamTotalDmgDifference,0.039690
RedTeamTotalGold,0.031556
RedTeamTotalDamageDoneToChampions,0.031206
BlueTeamKdaDifference,0.031177
BlueTeamTotalMinionsMonstersDifference,0.030388
BlueTeamXp,0.027731
RedTeamXp,0.025667


# Can We Keep Only One Team's Data and Maintain or Improve Model Accuracy?

In [48]:
cols_to_keep = X_train_team_data.columns[(X_train_team_data.columns.str.contains('Blue')) & (X_train_team_data.columns.str.contains('Difference'))]
cols_to_keep

Index(['BlueTeamTotalGoldDifference', 'BlueTeamLevelDifference',
       'BlueTeamXpDifference', 'BlueTeamWardDifference',
       'BlueTeamminionKillDifference', 'BlueTeamDeathsDifference',
       'BlueTeamMagicDmgDifference', 'BlueTeamPhysicalDmgDifference',
       'BlueTeamTrueDmgDifference', 'BlueTeamTotalDmgDifference',
       'BlueTeamTotalMinionsMonstersDifference', 'BlueTeamTimeCCingDifference',
       'BlueTeamWaterDragonDifference', 'BlueTeamAirDragonDifference',
       'BlueTeamChemtechDragonDifference', 'BlueTeamFireDragonDifference',
       'BlueTeamHextechDragonDifference', 'BlueTeamEarthDragonDifference',
       'BlueTeamKdaDifference'],
      dtype='object')

In [49]:
X_train_blue_team = X_train_team_data.loc[:, cols_to_keep]
X_train_blue_team

,BlueTeamTotalGoldDifference,BlueTeamLevelDifference,BlueTeamXpDifference,BlueTeamWardDifference,BlueTeamminionKillDifference,BlueTeamDeathsDifference,BlueTeamMagicDmgDifference,BlueTeamPhysicalDmgDifference,BlueTeamTrueDmgDifference,BlueTeamTotalDmgDifference,BlueTeamTotalMinionsMonstersDifference,BlueTeamTimeCCingDifference,BlueTeamWaterDragonDifference,BlueTeamAirDragonDifference,BlueTeamChemtechDragonDifference,BlueTeamFireDragonDifference,BlueTeamHextechDragonDifference,BlueTeamEarthDragonDifference,BlueTeamKdaDifference
3513,1046,6,2706,-173,10,-4,-774,1105,-418,-88,12,110674,0.0,0.0,0.0,1.0,0.0,0.0,14.25
1250,1327,2,2043,2,8,-3,-902,2903,385,2383,41,76997,0.0,0.0,0.0,0.0,0.0,0.0,9.75
3532,4519,3,3412,5,12,-7,-2636,1253,-206,-1590,35,-65221,0.0,0.0,0.0,0.0,1.0,0.0,17.25
3858,1433,1,1479,13,-8,-3,-583,4963,1515,5896,34,-24134,0.0,0.0,0.0,-1.0,0.0,0.0,5.25
528,834,1,1228,-181,-15,-2,-2197,2935,145,880,-24,-28320,0.0,0.0,0.0,-1.0,0.0,0.0,6.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,4619,3,3116,8,11,-8,2043,4113,464,6621,64,-31773,0.0,0.0,0.0,0.0,0.0,0.0,20.25
4060,-2111,-3,-2079,1,-4,7,2682,-5811,318,-2808,-44,28761,0.0,0.0,0.0,0.0,0.0,-1.0,-21.75
1346,250,1,1125,4,5,0,-524,622,-446,-348,-17,-5678,0.0,0.0,0.0,0.0,0.0,-1.0,-0.75
3454,1594,2,137,-1,4,0,252,-135,-560,-445,14,93964,1.0,0.0,0.0,0.0,0.0,0.0,2.25


In [52]:
#Build the models
best_model = model.get_random_forest_models(X_train_blue_team, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.6355
Max Depth:  7
Min Samples Per Leaf:  7


In [20]:
y_train.value_counts(normalize = True)

200.0    0.518509
100.0    0.481491
Name: winningTeam, dtype: float64

In [53]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train_blue_team.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

,0
BlueTeamTotalGoldDifference,0.168818
BlueTeamXpDifference,0.146776
BlueTeamTotalDmgDifference,0.104916
BlueTeamKdaDifference,0.080874
BlueTeamTotalMinionsMonstersDifference,0.080068
BlueTeamDeathsDifference,0.062566
BlueTeamMagicDmgDifference,0.053866
BlueTeamPhysicalDmgDifference,0.051670
BlueTeamTrueDmgDifference,0.049057
BlueTeamTimeCCingDifference,0.048864


# Can We Only Use Individual Stats and Maintain or Improve Model Accuracy?

In [22]:
cols_to_keep = X_train.columns[(X_train.columns.str.contains('Team') == False)& (X_train.columns.str.contains('dragon') == False)]
cols_to_keep

Index(['BlueTopAssists', 'RedSupportAssists', 'BlueJungleAssists',
       'BlueMidAssists', 'BlueADCAssists', 'BlueSupportAssists',
       'RedTopAssists', 'RedJungleAssists', 'RedMidAssists', 'RedADCAssists',
       ...
       'BlueMidKda', 'BlueBotKda', 'BlueSupportKda', 'RedTopKda',
       'RedJungleKda', 'RedMidKda', 'RedBotKda', 'RedSupportKda',
       'BlueJungleGankHeavy', 'RedJungleGankHeavy'],
      dtype='object', length=188)

In [23]:
X_train_players = X_train.loc[:, cols_to_keep]
X_train_players

,BlueTopAssists,RedSupportAssists,BlueJungleAssists,BlueMidAssists,BlueADCAssists,BlueSupportAssists,RedTopAssists,RedJungleAssists,RedMidAssists,RedADCAssists,...,BlueMidKda,BlueBotKda,BlueSupportKda,RedTopKda,RedJungleKda,RedMidKda,RedBotKda,RedSupportKda,BlueJungleGankHeavy,RedJungleGankHeavy
3513,1.0,0.0,1.0,1.0,4.0,2.0,1.0,0.0,0.0,1.0,...,2.25,3.25,1.75,-0.25,-0.75,0.25,-0.25,-0.50,200.0,100.0
1250,1.0,0.0,0.0,3.0,0.0,5.0,0.0,3.0,0.0,2.0,...,1.25,2.75,2.75,-1.00,0.25,2.75,0.50,-1.00,200.0,0.0
3532,1.0,1.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,...,1.25,3.50,2.75,-1.00,-1.00,-1.00,-0.75,-0.25,36.0,48.0
3858,1.0,6.0,3.0,1.0,0.0,4.0,1.0,0.0,0.0,4.0,...,2.25,2.25,0.00,-1.25,0.25,2.00,3.75,2.75,248.0,40.0
528,1.0,2.0,2.0,2.0,3.0,2.0,0.0,2.0,1.0,2.0,...,1.75,0.50,3.25,-1.50,1.75,0.00,5.25,0.00,160.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,3.0,2.0,1.0,0.0,1.0,5.0,0.0,2.0,1.0,2.0,...,5.25,4.75,3.00,-0.75,0.75,-1.75,-0.25,0.75,252.0,62.0
4060,2.0,5.0,1.0,0.0,1.0,1.0,0.0,6.0,2.0,0.0,...,-2.00,1.00,-1.00,-0.75,4.50,6.50,2.75,6.50,156.0,-44.0
1346,2.0,2.0,2.0,0.0,2.0,2.0,4.0,0.0,2.0,1.0,...,5.00,1.50,-0.50,1.25,6.50,-0.50,1.00,0.50,344.0,549.0
3454,0.0,0.0,1.0,0.0,2.0,3.0,0.0,1.0,0.0,2.0,...,0.00,3.00,1.50,1.25,4.50,0.00,-0.50,-2.00,36.0,240.0


In [24]:
#Build the models
best_model = model.get_random_forest_models(X_train_players, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.6185
Max Depth:  14
Min Samples Per Leaf:  5


In [25]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train_players.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

,0
BlueteamAssistDifference,0.016807
RedADCTotalGold,0.015036
gameDuration,0.013956
RedMidTotalGold,0.012236
BlueJungleXp,0.011413
RedTopTotalDamageDoneToChampions,0.010969
BlueTopMinionsKilled,0.010838
RedSupportXp,0.010798
BlueJungleTotalGold,0.010708
RedSupportTotalGold,0.010705


# Will a KNN Model Improve Accuracy?

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [27]:
#First need to scale the data
#Create the scaler
scaler = MinMaxScaler()

#Fit and transform on train data
X_train_scaled = scaler.fit_transform(X_train)

In [31]:
#Build a function for KNN
def get_KNN_models(X_train_scaled, y_train, param_dict, cv = 5):
    """
    This function takes in scaled data and builds an optimized KNN classification model. 
    It will use the parameters specified in the param_dict to optimizie across.
    
    This function utilizes GridSearchCV.
    
    This function returns the best model and prints out its parameters and mean
    cross-validated accuracy.
    """
    #Create the KNN model
    clf = KNeighborsClassifier()
    
    #Create the GridSearchCV object
    grid = GridSearchCV(clf, param_dict, cv = cv)
    
    #Fit the GridSearchCV object
    grid.fit(X_train_scaled, y_train)
    
    #Print the best model's score and parameters
    print('Mean Cross-Validated Accuracy: ', round(grid.best_score_, 4))
    print('Num Neighbors: ', grid.best_params_['n_neighbors'])
    print('Weights: ', grid.best_params_['weights'])
    
    #Return the best model
    return grid.best_estimator_

In [32]:
#Create a param_dict for KNN model
param_dict = {
    'n_neighbors': range(200, 500),
    'weights': ['uniform', 'distance']
}

In [33]:
#Test the function. Run it on the full X_train_scaled data set (Team and Player data)
best_model = get_KNN_models(X_train_scaled, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.6321
Num Neighbors:  300
Weights:  uniform


# Can We Use Only Team Data and Maintain or Improve KNN Model Accuracy?

In [34]:
#Scale the data
X_train_team_data_scaled = scaler.fit_transform(X_train_team_data)

In [35]:
#build the models
best_model = get_KNN_models(X_train_team_data_scaled, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.6355
Num Neighbors:  466
Weights:  uniform


# Can We Use Only A Single Team's Data and Maintain or Improve KNN Model Accuracy?

In [36]:
#Scale the data
X_train_blue_team_scaled = scaler.fit_transform(X_train_blue_team)

In [37]:
#Build the models
best_model = get_KNN_models(X_train_blue_team_scaled, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.6308
Num Neighbors:  280
Weights:  uniform


# Can We Only Use Individual Stats and Maintain or Improve KNN Model Accuracy?

In [38]:
#Scale the data
X_train_players_scaled = scaler.fit_transform(X_train_players)

In [39]:
#Build the models
best_model = get_KNN_models(X_train_players_scaled, y_train, param_dict, cv = 5)

Mean Cross-Validated Accuracy:  0.6123
Num Neighbors:  464
Weights:  uniform


# Can an AdaBoost Classifier Improve Model Accuracy?

In [42]:
#Set the params to optimize across
adaBoost_params = {
    'n_estimators': range(25, 101),
    'learning_rate': range(1, 11)
}

In [43]:
#Construct the model to use with adaBoost. Try Single Decision Tree
from sklearn.tree import DecisionTreeClassifier

model_to_boost = DecisionTreeClassifier(random_state = 123)

In [44]:
best_model = model.get_adaBoosted_model(X_train_blue_team, y_train, model_to_boost, adaBoost_params, cv = 5)

Mean Cross-Validated Accuracy:  0.5584
Num Estimators:  25
Learning Rate:  1
